# Project 1: Data analysis

## 1a) Bayesian approch

In order to predict the effect of genes on symptoms, we will use a simple Bayesian model. We consider these hypothesis:

$$\mu_0: P(symptoms|genes) = P(symptoms)$$
$$\mu_1: P(symptoms|genes) \neq P(symptoms)$$

In order words, does the genes have any effect on the symptoms or not? If it does, which genomes are the most relevant for the preventing the symptoms? 


In our case our Y is the symptoms and our X is the genes. Y is an 8 bit vector for each observation,with 1 or zero for each bit. X is an 128 with the same charectaristic as Y.


The probability distribution of $ Y $ depends on an unknown parameter which also is stocastic. In this case the unknown parameter is $\theta$ or $ p $. 

The formulation under is an alternative formulation of the the previous hypotesis.Does the parameter that the probability dist of Y depend on, depend on what the value of X is or not. $ \mu_0 $ says that it does not.$\mu_1 $ says it does.
 
$$\mu_0: \theta^0 \sim Beta(\alpha^0 , \beta^0) ,  \ \ \  Y/ \theta^0  $$
$$\mu_1: \theta^{1,x_t} \sim Beta(\alpha^{1,x_t} , \beta^{1,x_t}) , \ \ \ y_t/x_t \sim bernulli(\theta^{1,x_t}) $$
 

We will model the hypothesis in the following way. Given the data, we try to estimate

$$ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_1}(D) \cdot \xi(\mu_1)} $$

We can eather set $P(\mu | D)$ to mean $P(\mu_1 | D)$ or  $P(\mu_0 | D)$ depending on which hypothesis we want to check the posterior belief of. 

where $P_{\mu} (D) = P(\mu |D)$, $D = (x_t, y_t,x_{t-1},y_{t-1})_{t=1} ^T $ for a spesific gene, $x_t$ is the value of the gene for observation t ,$x_{t-1}$ is a vector of all the previous  values of the previous observation up to t-1,  $y_t$ is the the value of symptom Y for observation t,$y_{t-1}$ is a vector of all the previous  values of the previous symtoms observation up to t-1 and $\mu$ is one of the hypothesis in the set {$\mu_1$,$\mu_0$}.  We assume that the different observations(rows) of the data are independent of each other. This gives us:
$$ P_{\mu}(D) = P(y_1,...,y_t \cup x_1,...,x_t)$$

$$ = \prod_{i=1}^{t} P(y_i | x_i,x_{i-1} , y_{i-1}) $$


With these assumptions, we can estimate the terms in the $P_{\mu} (D)$ by 
$$ P(y_t | x_t,x_{t-1} , y_{t-1}) = \int_{\theta} P_{\theta}(y_t|x_t) \cdot \xi(\theta|x_{t-1},y_{t-1})$$

We calculate the likehood of adding a new observation(row) using this formula.It works if we assume that the the next observation is independent from the previous observations.


This is the marginal likelihood(posterior predictive distribution) which is a compound distribution,which is used to calculate $P(y_t | x_{t},x_{t-1}, y_{t-1})$. The probabilty $P(y_t | x_{t})$ is stocatic in $\theta$,so we have to integrate over all values of $\theta$. Every time a new observation is added we calculate $P(y_t | x_{t},x_{t-1}, y_{t-1})$,and recalculate $$ = \prod_{i=1}^{t} P(y_i | x_i ,x_{i-1}, y_{i-1}) $$. 

So to calculate $$ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_i}(D) \cdot \xi(\mu_i)} $$,we are updating our belief,every time a new observation is added,sequentially. For every update $\xi(\mu)$ is $P(\mu | D)$ from the previous iteration,exept the first iteration where $\xi(\mu)$ cancels out because it's constant.

This integral simplifies to:

$$ P(y_t | x_{t},x_{t-1},y^{t-1}) = \int_{\theta} P_{\theta}(y_t) \cdot d \xi(\theta|y^{t-1}) = \frac{\alpha_{t,x_t}}{\alpha_{t,x_t} + \beta_{t,x_t}}$$


Looking at $\mu_1$ we are using the beta-binomial distribution.Its a binomial distribution where  the probability of success at each of n trials is not fixed but comes from a beta distribution. The bernulli distribtuion is a special case of the binomial distribution,where number of trials is equal to 1.

We are doing this because our y's are either 1 or 0.Then it is reasonable to assume that $Y|X$ is bernulli distributed.

It is also conviniant because of its conjugate prior property.Now we can calculate  $P( D/\mu_1) =\prod_{i=1}^{t} P(y_i | x^i ,x^{i-1}, y^{i-1}) $ beacuse we know that the marginal likelihood (posterior predictive dist) is composed of a binomial dist times a beta dist $ P(y_t | x^t , y^{t-1}) = \int_{\theta} P_{\theta}(y_t|x_t) \cdot \xi(\theta|x^{t-1},y^{t-1})$,which is beta distributed .For each iteration we end up  getting that $P_{\mu} (D)$ is beta distributed.

The same integral calculation is done for $P(D | \mu_0)$ but now the marginal likelihood is independent of the data.

Instead of calculating $P(  D|\mu_1) =\prod_{i=1}^{t} P(y_i | x_i ,x_{i-1}, y_{i-1})$ ,we can first calculate $log(P(  D|\mu_1))$ and then do $e^{log(P(  D|\mu_1))}$,same for $P(D | \mu_0)$.

To summarize.We have a set of hypotesis in this case two.We want to find how accurate they are. So we have a prior beleif on the strenght of the hypothses which we express through a probability, and we want to update our belief on the strenght of the hypotesis(posterior belief). In other word what is the probability of the choosen hypotesis given the data.Ony have to compute one of the probabilities because $P(\mu_1 | D)=1-P(\mu_0 | D)$.

The calculation is done sequentially.For each iteration we get a new data observation.A new row of genes.We continously update the posterior belief,for all genes,and put the posterior belief of each gene into a list. The belief get updated through calculating the marginal likelihood every time a new row is added and a product is taken between this marginal likelihood and all the presious marginal likelihood calculations bought for $P(  D|\mu_1)$ and for$P(  D|\mu_0)$ this is then put into the bayes formula $ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_i}(D) \cdot \xi(\mu_i)} $.

We want to find this probability for for each of the genes.This procedure is done for each gene.For each gene if the posterior probabilty is greater then some trechhold we add this gene to the list of important genes that are relvant for explaining that spesific symptom.
 



The decision rule can be defined as

$$ \mu =   \left\{
\begin{array}{ll}
      \mu_1 & P(\mu_1|D) > s \\
      \mu_0 & else
\end{array} 
\right.  $$

where $s$ is a given threshold.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [5]:
cols = (['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death', 
        'Age', 'Gender', 'Income'] + 
         [f'Gene_{i+1:03}' for i in range(128)] + 
         ['Asthma', 'Obesity', 'Smoking', 'Diabetes', 'Heart disease', 'Hypertension',
         'Vacc_1', 'Vacc_2', 'Vacc_3'])


In [52]:
obs_data = pd.read_csv("observation_features.csv")
treat_data = pd.read_csv("treatment_features.csv") 
action_data = pd.read_csv("treatment_actions.csv")
outcome_data = pd.read_csv("treatment_outcomes.csv") 

obs_data.columns = cols
treat_data.columns = cols
outcome_data.columns = cols[:10]
action_data.columns = ['Treatment_1', 'Treatment_2']

In [7]:
symptoms = np.array(obs_data.iloc[:,0:10])
age = obs_data.iloc[:,10]
gender = obs_data.iloc[:,11]
income = obs_data.iloc[:,12]
genome = obs_data.iloc[:,13:141]
comorbidities = obs_data.iloc[:,141:147]
vaccination_status = np.array(obs_data.iloc[:,147:])

In [8]:
def select_features(X, Y, threshold):
    n_features = X.shape[1]
    n_data =  X.shape[0]
    alpha_b = np.ones([n_features, 2 ])
    beta_b = np.ones([n_features, 2])
    log_p = np.zeros(n_features)
    
    log_null = 0
    alpha = 1
    beta = 1  
    for t in range(n_data):
        p_null = alpha / (alpha + beta)
        log_null += np.log(p_null)*Y[t] + np.log(1-p_null)*(1 - Y[t])
        alpha += Y[t]
        beta += (1 - Y[t])
        for i in range(n_features):
            
                x_ti = int(X[t,i])
                p = alpha_b[i, x_ti] / (alpha_b[i, x_ti] + beta_b[i, x_ti])
                log_p[i] += np.log(p)*Y[t] + np.log(1-p)*(1 - Y[t])
                alpha_b[i, x_ti] += Y[t]
                beta_b[i, x_ti] += (1 - Y[t])
    log_max=np.mean(log_p)
    log_max2=np.mean(log_null)
    log_p=log_p-log_max
    log_null=log_null-log_max2
    p = np.exp(log_p) / (np.exp(log_p) + np.exp(log_null))
    #print((np.exp(log_p) + np.exp(log_null)))

    features = []
    for i in range(n_features):
        if p[i] > threshold:
            features.append(i)
    print(alpha)
    print(beta)
    return features

In [9]:
select_0 = select_features(np.array(genome),symptoms[:,1],0.8)

22029.0
77972.0


In [10]:
select_0

[3,
 14,
 17,
 26,
 27,
 30,
 37,
 38,
 39,
 40,
 41,
 45,
 48,
 57,
 65,
 67,
 68,
 72,
 85,
 89,
 90,
 100,
 105,
 111,
 120,
 121,
 123,
 126]

## 1b) Estimating the efficacy of vaccines

We estimate the efficacy of the vaccines by
$$ 100 \cdot (1-IRR)$$
where IRR is the ratio of the vaccinated incidence rate and not vaccinated incidence incidenes ([link](https://www.nejm.org/doi/full/10.1056/NEJMoa2034577)). This means that the relationship we model for each symptom is as given below:

$r_v = P(symptom = 1 | vaccine = 1)$ and

$r_n = P(symptom = 1 | vaccine = 0)$

to obtain $IRR = 100 * [1 - r_v / r_n]$

We obtain a Bayesian estimate of $r_v, r_n$
through the Beta-Bernoulli conjugate prior.

As in the abovementioned article, we use a Bayesian Beta-Binomial model. Hence, our prior belief is assumed to have a Beta distribution and the Likelihood of observed data is Binomial distributed. This results in a Beta-distributed posterior (more details will follow afterwards).

We set a low beta value for the prior distribution because we are uncertain about the actual distribution and compute a mean prior probability of each symptom from the data. Given the beta value of 1 and the mean probability, we compute the alpha value using the function alpha = beta \* p/(1-p), which for p in (0, 0.5] returns an alpha in (0, 1]. If p is 0.5 this would make the beta distribution uniform, with alpha = beta = 1. 

In [19]:
def count_status(vac_list, symptom_name):
    index = symptom_names.index(symptom_name)
    vacc_pos = 0
    for i in vac_list:
        if symptoms[i][1] == 1 and symptoms[i][index] == 1: 
            vacc_pos += 1
    return vacc_pos

def find_alpha(beta,p):
    return beta*p/(1-p)

def find_efficacy(vaccinated,not_vaccinated,symptom_name,count_status):
    vacc_pos = count_status(vaccinated,symptom_name)
    not_pos = count_status(not_vaccinated,symptom_name)

    v = vacc_pos/len(vaccinated)
    n_v = not_pos/len(not_vaccinated)
    IRR = v/n_v
    
    efficacy = 100*(1- IRR)
    
    N = len(obs_data)
    beta = 1
    p = prior_probs[symptom_name]
    alpha = find_alpha(beta,p)
    
    samples_vaccine = stats.beta.rvs(alpha + vacc_pos, beta + len(vaccinated) - vacc_pos, size=N)
    samples_no_vaccine = stats.beta.rvs(alpha + not_pos, beta + len(not_vaccinated) - not_pos, size=N)
    
    samples_ve = 100 * (1 - samples_vaccine/samples_no_vaccine)
    lower = np.percentile(samples_ve, 2.5)
    upper = np.percentile(samples_ve, 97.5)
    print(f'{symptom_name:15s}: {efficacy:3.3f} - ({lower:3.3f}, {upper:3.3f})')    

In [20]:
#samples with vaccinations or not:
not_vaccinated = [i for i in range(len(vaccination_status)) if all(v==0 for v in vaccination_status[i])]
vaccinated = [i for i in range(len(vaccination_status)) if any(v !=0 for v in vaccination_status[i])]
symptom_names = ['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death']
prior_probs= {key:np.sum(obs_data.iloc[:,i]) / len(obs_data) for i, key in enumerate(symptom_names)}

print("Efficacy of the vaccinated:")
for s in symptom_names:
    find_efficacy(vaccinated,not_vaccinated,s,count_status)
    


Efficacy of the vaccinated:
Covid-Recovered: 28.965 - (20.078, 36.889)
Covid-Positive : 21.464 - (19.596, 23.273)
No-Taste/Smell : 47.521 - (41.638, 52.870)
Fever          : 51.575 - (44.922, 57.441)
Headache       : 45.473 - (28.882, 58.312)
Pneumonia      : 57.810 - (51.591, 63.337)
Stomach        : 50.542 - (25.722, 67.486)
Myocarditis    : 44.531 - (28.451, 57.151)
Blood-Clots    : 57.190 - (50.898, 62.806)
Death          : 93.394 - (90.544, 95.663)


In [21]:
def find_efficacy(group_pos: pd.DataFrame, group_neg: pd.DataFrame, symptom_index: int, prior_probs):
    group_pos_count = np.sum(group_pos.iloc[:,symptom_index] * group_pos.iloc[:,1])
    group_neg_count = np.sum(group_neg.iloc[:,symptom_index] * group_neg.iloc[:,1])

    v = group_pos_count/len(group_pos)
    n_v = group_neg_count/len(group_neg)
    
    if n_v == 0:
        print('Division by zero')
        return
    
    IRR = v/n_v
    
    #print(v, n_v)
    efficacy = 100*(1- IRR)
    
    N = len(group_pos) + len(group_neg)
    beta = 1
    p = prior_probs[symptom_index]
    alpha = find_alpha(beta,p)
    
    symptom_name = symptom_names[symptom_index]
    
    samples_group_pos = stats.beta.rvs(alpha + group_pos_count, beta + len(group_pos) - group_pos_count, size=N)
    samples_group_neg = stats.beta.rvs(alpha + group_neg_count, beta + len(group_neg) - group_neg_count, size=N)
    
    samples_ve = 100 * (1 - samples_group_pos/samples_group_neg)
    lower = np.percentile(samples_ve, 2.5)
    upper = np.percentile(samples_ve, 97.5)
    print(f'{symptom_name:15s}: {efficacy:3.3f} - ({lower:3.3f}, {upper:3.3f})')    

In [22]:
obs_data.iloc[:,9] * obs_data.iloc[:,1]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99994    0.0
99995    0.0
99996    0.0
99997    0.0
99998    0.0
Length: 99999, dtype: float64

In [24]:
#samples with vaccinations or not:
vacced = obs_data[np.sum(obs_data.iloc[:,-3:], axis=1) == 1]
vacced_neg = vacced[vacced.iloc[:,1]==0]
vacced_pos = vacced[vacced.iloc[:,1]==1]

un_vacced = obs_data[np.sum(obs_data.iloc[:,-3:], axis=1) == 0]
un_vacced_neg = un_vacced[un_vacced.iloc[:,1]==0]
un_vacced_pos = un_vacced[un_vacced.iloc[:,1]==1]

symptom_names = ['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death']
prior_probs= [np.sum(obs_data.iloc[:,i]) / len(obs_data) for i, key in enumerate(symptom_names)]

print("Efficacy of the vaccinated:")
for i, s in enumerate(symptom_names):
    find_efficacy(vacced,un_vacced,i,prior_probs)
    

Efficacy of the vaccinated:
Covid-Recovered: 28.965 - (20.126, 36.877)
Covid-Positive : 21.464 - (19.606, 23.280)
No-Taste/Smell : 47.521 - (41.595, 52.830)
Fever          : 51.575 - (44.890, 57.452)
Headache       : 45.473 - (28.992, 58.390)
Pneumonia      : 57.810 - (51.559, 63.323)
Stomach        : 50.542 - (26.091, 67.392)
Myocarditis    : 44.531 - (28.456, 57.078)
Blood-Clots    : 57.190 - (50.948, 62.748)
Death          : 93.394 - (90.527, 95.672)


In [26]:
vacc_type1 = obs_data[obs_data.Vacc_1 == 1]
vacc_type2 = obs_data[obs_data.Vacc_2 == 1]
vacc_type3 = obs_data[obs_data.Vacc_3 == 1]
vaccination_types = [vacc_type1,vacc_type2,vacc_type3]
vaccination_names = ['type 1', 'type 2', 'type 3']

for name in vaccination_names:
    print(name)
    index = vaccination_names.index(name)
    for i, s in enumerate(symptom_names):
        find_efficacy(vaccination_types[index],un_vacced,i,prior_probs)
    print()

type 1
Covid-Recovered: 19.633 - (6.103, 31.678)
Covid-Positive : 11.296 - (8.514, 14.007)
No-Taste/Smell : 34.419 - (24.420, 43.292)
Fever          : 43.590 - (32.905, 52.995)
Headache       : 30.767 - (2.812, 52.363)
Pneumonia      : 54.129 - (44.076, 62.694)
Stomach        : 28.704 - (-17.295, 59.510)
Myocarditis    : 36.093 - (10.668, 55.746)
Blood-Clots    : 48.746 - (38.272, 57.921)
Death          : 95.704 - (91.559, 98.419)

type 2
Covid-Recovered: 28.837 - (16.301, 39.749)
Covid-Positive : 20.584 - (17.975, 23.131)
No-Taste/Smell : 43.109 - (34.088, 51.294)
Fever          : 45.703 - (35.230, 54.945)
Headache       : 39.907 - (14.296, 59.387)
Pneumonia      : 48.345 - (37.816, 57.625)
Stomach        : 70.324 - (42.793, 87.560)
Myocarditis    : 35.284 - (9.560, 54.926)
Blood-Clots    : 47.762 - (37.230, 57.046)
Death          : 92.922 - (87.717, 96.637)

type 3
Covid-Recovered: 38.211 - (26.732, 48.330)
Covid-Positive : 32.271 - (29.915, 34.600)
No-Taste/Smell : 64.686 - (57.867,

## 1c) Estimating the probability of vaccination side-effects.

Using the Bayes theorem, we will estimate the probability of vaccination side-effects. In other words, the probability of getting symptoms given that a person is vaccinated and has tested negative.
$$p_1 = P(symptom | vaccine,covid') = \frac{P(symptom \cap vaccine,covid')}{P(vaccine,covid')} $$
$$p_2 = P(symptom | vaccine',covid') = \frac{P(symptom \cap vaccine',covid')}{P(vaccine',covid')} $$

We want to test the following hypothesis:
$$ h_0: p_1 - p_2 \leq 0 $$
$$h_a: p_1 - p_2 > 0 $$

We make a confidence intervall for each symptom, and reject the null hypothesis if confidence intervall does not include zero. 

In [46]:
vacced = obs_data[np.sum(obs_data.iloc[:,-3:], axis=1) == 1]
vacced_neg = vacced[vacced.iloc[:,1]==0]

un_vacced = obs_data[np.sum(obs_data.iloc[:,-3:], axis=1) == 0]
un_vacced_neg = un_vacced[un_vacced.iloc[:,1]==0]

for i in range(2, 10):
    p1 = vacced_neg.sum()[i] / len(obs_data) / (len(vacced_neg) / len(obs_data))
    p2 = un_vacced_neg.sum()[i] / len(obs_data) / (len(un_vacced_neg) / len(obs_data))
    
    lower = (p1-p2 - 1.64 * np.sqrt((p1*(1-p1) / len(vacced_neg)) + (p2 * (1-p2) / len(un_vacced_neg))))
    higher = (p1-p2 + 1.64 * np.sqrt((p1*(1-p1) / len(vacced_neg)) + (p2 * (1-p2) / len(un_vacced_neg))))

    print(f'{symptom_names[i]:15s}:{p1:4.5f}',end=' ')
    print(f'{p2:4.5f}',end=' ')
    print(f' diff: {p1-p2:8.5f} ({lower:8.5f},{higher:6.5f}) {"rejected" if lower>0 else "null_hypothesis"}')

No-Taste/Smell :0.00081 0.00057  diff:  0.00024 (-0.00007,0.00055) null_hypothesis
Fever          :0.09903 0.00508  diff:  0.09395 ( 0.09161,0.09628) rejected
Headache       :0.05579 0.01053  diff:  0.04526 ( 0.04329,0.04723) rejected
Pneumonia      :0.00133 0.00144  diff: -0.00011 (-0.00056,0.00035) null_hypothesis
Stomach        :0.00231 0.00257  diff: -0.00026 (-0.00086,0.00034) null_hypothesis
Myocarditis    :0.00214 0.00047  diff:  0.00168 ( 0.00127,0.00208) rejected
Blood-Clots    :0.00243 0.00110  diff:  0.00133 ( 0.00085,0.00182) rejected
Death          :0.00035 0.00000  diff:  0.00035 ( 0.00021,0.00049) rejected


**A large-sample 95% confidence interval for $p_1 - p_2$**:

In [47]:
vac_probs = [(vacced_neg.sum()[i] / len(obs_data)) / (len(vacced_neg) / len(obs_data)) for i in range(2,10)]
unvac_probs = [(un_vacced_neg.sum()[i] / len(obs_data)) / (len(un_vacced_neg) / len(obs_data)) for i in range(2,10)]
list(zip(vac_probs, unvac_probs))
np.array(vac_probs) - np.array(unvac_probs)

i = 2
for p1, p2 in zip(vac_probs, unvac_probs):
    lower = (p1-p2 - 1.64 * np.sqrt((p1*(1-p1) / len(vacced_neg)) + (p2 * (1-p2) / len(un_vacced_neg))))
    higher = (p1-p2 + 1.64 * np.sqrt((p1*(1-p1) / len(vacced_neg)) + (p2 * (1-p2) / len(un_vacced_neg))))
    print(f"{symptom_names[i]:15s}: ({lower:8.5f},{higher:6.5f})")
    i += 1

No-Taste/Smell : (-0.00007,0.00055)
Fever          : ( 0.09161,0.09628)
Headache       : ( 0.04329,0.04723)
Pneumonia      : (-0.00056,0.00035)
Stomach        : (-0.00086,0.00034)
Myocarditis    : ( 0.00127,0.00208)
Blood-Clots    : ( 0.00085,0.00182)
Death          : ( 0.00021,0.00049)


In [53]:
treat_data

,Covid-Recovered,Covid-Positive,No-Taste/Smell,Fever,Headache,Pneumonia,Stomach,Myocarditis,Blood-Clots,Death,...,Gene_128,Asthma,Obesity,Smoking,Diabetes,Heart disease,Hypertension,Vacc_1,Vacc_2,Vacc_3
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
873,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
874,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [59]:
outcome_data.sum(), treat_data.sum()[:10]

(Covid-Recovered     90.0
 Covid-Positive     877.0
 No-Taste/Smell      23.0
 Fever               18.0
 Headache             1.0
 Pneumonia           19.0
 Stomach              3.0
 Myocarditis          6.0
 Blood-Clots         17.0
 Death                8.0
 dtype: float64,
 Covid-Recovered     90.0
 Covid-Positive     877.0
 No-Taste/Smell      49.0
 Fever               24.0
 Headache             7.0
 Pneumonia           34.0
 Stomach              5.0
 Myocarditis         13.0
 Blood-Clots         34.0
 Death               10.0
 dtype: float64)

In [69]:
new_treat_data = treat_data.copy()[((np.sum(treat_data.iloc[:,2:10],axis=1) > 0.0) 
              | np.sum(outcome_data.iloc[:,2:10],axis=1) > 0.0)]
new_treat_data

,Covid-Recovered,Covid-Positive,No-Taste/Smell,Fever,Headache,Pneumonia,Stomach,Myocarditis,Blood-Clots,Death,...,Gene_128,Asthma,Obesity,Smoking,Diabetes,Heart disease,Hypertension,Vacc_1,Vacc_2,Vacc_3
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
19,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
863,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
864,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
866,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [73]:
group_first = new_treat_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 0))]
group_second = new_treat_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 1))]
group_both = new_treat_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 1))]
group_none = new_treat_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 0))]

C:\Users\Tellef\AppData\Local\Temp/ipykernel_16072/4289214852.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  group_first = new_treat_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 0))]
C:\Users\Tellef\AppData\Local\Temp/ipykernel_16072/4289214852.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  group_second = new_treat_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 1))]
C:\Users\Tellef\AppData\Local\Temp/ipykernel_16072/4289214852.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  group_both = new_treat_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 1))]
C:\Users\Tellef\AppData\Local\Temp/ipykernel_16072/4289214852.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  group_none = new_treat_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 0))]


The relation we are modelling is whether any of the individuals who had symptoms do not have those symptoms after treatment or got symptoms from the treatment. 

We are not interested in the outcomes for individuals who did not have any symptoms neither before nor after the treatment.

We do this modelling task in a similar way that we did the modelling for vaccine efficacy. Since we now have data from before and after a treatment, we calculate the ratio of people who still have symptoms after treatment to the people who had symptoms before treatment for both the treated group and the control group. 

We estimate the efficacy of the treatments by
$$ 100 \cdot (1-IRR)$$
where IRR is the ratio of the treated incidence rate and not treated incidence rate ([link](https://www.nejm.org/doi/full/10.1056/NEJMoa2034577)). This means that the relationship we model for each symptom is as given below:

$t_1 = P(symptom = 1 | treatment = 1)$ after treatment, $t_0 = P(symptom = 1 | treatment = 1) $ before treatment.

$n_1 = P(symptom = 1 | treatment = 0)$ after treatment, $n_0 = P(symptom = 1 | treatment = 0) $ before treatment

$r_t = t_1 / t_0$ and

$r_n = n_1 / n_0$

to obtain $IRR = 100 * [1 - r_t / r_n]$

In [170]:
new_outcome_data = outcome_data.copy()[((np.sum(treat_data.iloc[:,2:10],axis=1) > 0.0) 
              | np.sum(outcome_data.iloc[:,2:10],axis=1) > 0.0)]

In [171]:
outcome_first = new_outcome_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 0))]
outcome_second = new_outcome_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 1))]
outcome_both = new_outcome_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 1))]
outcome_none = new_outcome_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 0))]

C:\Users\Tellef\AppData\Local\Temp/ipykernel_16072/4031450266.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  outcome_first = new_outcome_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 0))]
C:\Users\Tellef\AppData\Local\Temp/ipykernel_16072/4031450266.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  outcome_second = new_outcome_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 1))]
C:\Users\Tellef\AppData\Local\Temp/ipykernel_16072/4031450266.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  outcome_both = new_outcome_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 1))]
C:\Users\Tellef\AppData\Local\Temp/ipykernel_16072/4031450266.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  outcome_none = new_outcome_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 0))]


In [205]:
def treatment_efficacy(outcome_treated, precondition_treated, outcome_untreated, precondition_untreated, prior_probs, symptom_name):
    group_pos_count = np.sum(outcome_treated[symptom_name])
    group_neg_count = np.sum(outcome_untreated[symptom_name])
    
    group_pos_total = np.sum(precondition_treated[symptom_name])
    group_neg_total = np.sum(precondition_untreated[symptom_name])
    
    if any(v == 0 for v in (group_pos_total, group_neg_total, group_neg_count)):
        print(f'{symptom_name:15s}: Division by zero - not enough data to compute efficacy' )
        return
    
    v = group_pos_count / group_pos_total
    n_v = group_neg_count / group_neg_total

    #print(f"{group_pos_count=}, {group_pos_total=}\n {group_neg_count=} {group_neg_total=}\n, {v=}, {n_v=}")

    IRR = v/n_v

    efficacy = 100*(1- IRR)

    N = len(precondition_treated) + len(precondition_untreated)
    beta = 1
    p = prior_probs[4]
    alpha = find_alpha(beta,p)

    #symptom_name = symptom_names[symptom_index]

    samples_group_pos = stats.beta.rvs(alpha + group_pos_count, beta + len(outcome_treated) - group_pos_count, size=N)
    samples_group_neg = stats.beta.rvs(alpha + group_neg_count, beta + len(outcome_untreated) - group_neg_count, size=N)

    samples_ve = 100 * (1 - samples_group_pos/samples_group_neg)
    lower = np.percentile(samples_ve, 2.5)
    upper = np.percentile(samples_ve, 97.5)
    print(f'{symptom_name:15s}: {efficacy:7.3f} - 95% CI: ({lower:3.3f}, {upper:3.3f})')    

In [197]:
prior_probs= [(np.sum(new_treat_data[sym]) + np.sum(new_outcome_data[sym])) / (len(new_treat_data) * 2) for sym in symptom_names]

In [198]:
prior_probs

[0.04878048780487805,
 1.0,
 0.21951219512195122,
 0.12804878048780488,
 0.024390243902439025,
 0.16158536585365854,
 0.024390243902439025,
 0.057926829268292686,
 0.15548780487804878,
 0.054878048780487805]

In [199]:
treatment_efficacy(outcome_first, group_first, outcome_none, group_none, prior_probs, "Blood-Clots")

Blood-Clots    : 100.000 95% CI: (97.970, 100.000)


In [200]:
treatment_efficacy(outcome_second, group_second, outcome_none, group_none, prior_probs, "Myocarditis")

Myocarditis    :  75.000 95% CI: (-9774.589, 95.667)


In [201]:
treatment_efficacy(outcome_both, group_both, outcome_none, group_none, prior_probs, "Death")

Death          :  50.000 95% CI: (-1799.239, 97.661)


In [207]:
for outcome_treated, pre_treated, treatment in zip([outcome_first, outcome_second, outcome_both], 
                                        [group_first, group_second, group_both],
                                                ['treatment 1', 'treatment 2', 'both treatments']):
    print(f"{treatment} efficacy:")
    for key in outcome_data.keys()[2:]:
        #print(key)
        treatment_efficacy(outcome_treated, pre_treated, outcome_none, group_none, prior_probs, key)
    print()

treatment 1 efficacy:
No-Taste/Smell :  33.333 - 95% CI: (-8.415, 70.220)
Fever          :  40.000 - 95% CI: (-60.550, 75.847)
Headache       : 100.000 - 95% CI: (43.231, 100.000)
Pneumonia      :  80.000 - 95% CI: (0.038, 96.339)
Stomach        : Division by zero - not enough data to compute efficacy
Myocarditis    :   0.000 - 95% CI: (-6302.088, 30.167)
Blood-Clots    : 100.000 - 95% CI: (97.878, 100.000)
Death          :   0.000 - 95% CI: (-1069.492, 65.480)

treatment 2 efficacy:
No-Taste/Smell :  71.429 - 95% CI: (48.740, 96.492)
Fever          :  25.000 - 95% CI: (-52.284, 86.853)
Headache       : 100.000 - 95% CI: (17.290, 100.000)
Pneumonia      :   0.000 - 95% CI: (-127.399, 60.050)
Stomach        :  33.333 - 95% CI: (-8803.295, 91.040)
Myocarditis    :  75.000 - 95% CI: (-1066.744, 93.806)
Blood-Clots    :  46.154 - 95% CI: (-47.585, 75.029)
Death          :  50.000 - 95% CI: (-159.547, 97.703)

both_treatments efficacy:
No-Taste/Smell : 100.000 - 95% CI: (97.887, 100.000)
Fe

In [166]:
outcome_both.sum(), group_both.sum()[:10]

(Covid-Recovered     2.0
 Covid-Positive     39.0
 No-Taste/Smell      0.0
 Fever               2.0
 Headache            0.0
 Pneumonia           0.0
 Stomach             0.0
 Myocarditis         0.0
 Blood-Clots         0.0
 Death               1.0
 dtype: float64,
 Covid-Recovered     2.0
 Covid-Positive     39.0
 No-Taste/Smell     17.0
 Fever               3.0
 Headache            2.0
 Pneumonia           7.0
 Stomach             1.0
 Myocarditis         4.0
 Blood-Clots         6.0
 Death               2.0
 dtype: float64)